In [3]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
import pandas as pd
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from collections import defaultdict
import spacy
import spacy
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn import metrics
import optuna
from sklearn.svm import SVC
from sklearn.utils import resample
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import KeyedVectors
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\didia\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\didia\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "c:\Users\didia\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 711, in start
    self.io_loop.start()
  File "c:\Users\didia\anaconda3\Lib\site-packa

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

  Obtaining dependency information for matplotlib from https://files.pythonhosted.org/packages/33/cd/b145f9797126f3f809d177ca378de57c45413c5099c5990de2658760594a/matplotlib-3.10.7-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
    --------------------------------------- 0.2/8.1 MB 4.6 MB/s eta 0:00:02
    --------------------------------------- 0.2/8.1 MB 2.0 MB/s eta 0:00:05
   - -------------------------------------- 0.3/8.1 MB 2.5 MB/s eta 0:00:04
   - -------------------------------------- 0.4/8.1 MB 2.5 MB/s eta 0:00:04
   -- ------------------------------------- 0.4/8.1 MB 2.0 MB/s eta 0:00:04
   -- ------------------------------------- 0.5/8.1 MB 2.1 MB/s eta 0:00:04
   -- ------------------------------------- 0.5/8.1 MB 2.1 MB/s eta 0:00:04
   --- ------------------------------------ 0.7/8.1 MB 1.8 MB/s eta 0:00:05
   --- ------------------------------------ 0.7/8.1 MB 1.9 MB/s eta 0:00:04
   --- -------------------------

In [118]:
#carregar dataset
df = pd.read_csv('factnews_dataset.csv')
df.isnull().sum()

file          0
id_sente      0
id_article    0
domain        0
year          0
sentences     0
classe        0
dtype: int64

# Pré-processamento

divisão dos dados

In [ ]:
'''
#criar grupo treino, validação e teste
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df['classe'])
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['classe'])

#guardar conjuntos em memória
train_df.to_csv("train.csv", index=False) 
val_df.to_csv("val.csv", index=False)
test_df.to_csv("test.csv", index=False)
'''

balanceamento dos dados

In [119]:
train= pd.read_csv('train.csv')
val= pd.read_csv('val.csv')
test= pd.read_csv('test.csv')

y_test= test['classe']
y_train= train['classe']
y_val=val['classe']
classes = test['classe'].unique()

In [120]:
# Contagem das classes em cada conjunto
print ("conjunto de treino: ",y_train.value_counts().sort_index())
print("\nconjunto de validação: ",y_val.value_counts().sort_index())
print("\nconjunto de teste: ", y_test.value_counts().sort_index())



conjunto de treino:  classe
-1     779
 0    2375
 1     312
Name: count, dtype: int64

conjunto de validação:  classe
-1    195
 0    594
 1     78
Name: count, dtype: int64

conjunto de teste:  classe
-1     417
 0    1273
 1     168
Name: count, dtype: int64


In [ ]:
'''from sklearn.utils import resample

# Separar cada classe
class_min = train[train['classe'] == -1]
class_med = train[train['classe'] == 1]
class_max = train[train['classe'] == 0]

# Número da classe maior
n_samples = len(class_max)

# Replicar as classes menores até o tamanho da maior
class_min_upsampled = resample(class_min, 
                            replace=True, 
                            n_samples=n_samples, 
                            random_state=42)

class_med_upsampled = resample(class_med, 
                            replace=True, 
                            n_samples=n_samples, 
                            random_state=42)

train_oversampling = pd.concat([class_min_upsampled, class_med_upsampled, class_max])

# baralhar o conjunto de treino
train_oversampling = train.sample(frac=1, random_state=42).reset_index(drop=True)

print(train_oversampling['classe'].value_counts())
train_oversampling.to_csv('train_oversampling.txt', index=False)
'''


classe
 0    2375
-1     779
 1     312
Name: count, dtype: int64


tokeinização

In [124]:
stop_words = set(stopwords.words('portuguese'))

#criar coluna tokens
train['tokens'] = train['sentences'].apply(lambda x: nltk.word_tokenize(str(x)))
val['tokens']   = val['sentences'].apply(lambda x: nltk.word_tokenize(str(x)))
test['tokens'] = test['sentences'].apply(lambda x: nltk.word_tokenize(str(x)))

#remover stopwords
train['tokens'] = train['tokens'].apply(lambda toks: [t for t in toks if t.lower() not in stop_words])
val['tokens']   = val['tokens'].apply(lambda toks: [t for t in toks if t.lower() not in stop_words])
test['tokens']  = test['tokens'].apply(lambda toks: [t for t in toks if t.lower() not in stop_words])

#matizar
#nltk . download ('rslp')
stemmer = nltk . stem . RSLPStemmer ()
train['stems'] = train['tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])
val['stems']   = val['tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])
test['stems'] = test['tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])

nltk . download ('rslp')
stemmer = nltk . stem . RSLPStemmer ()
train['stems'] = train['tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])
print(train['stems'].head())

y_test= test['classe']
y_train= train['classe']
y_val=val['classe']
classes = test['classe'].unique()


[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\didia\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


0    [neym, ,, part, 20, novembr, vai, lider, seleç...
1    [forç, internac, transform, quest, taiwan, ass...
2    [alianç, mai, quant, parlament, eleit, nome, p...
3    [ger, mort, '', ,, declar, cabr, ,, 16ª, entre...
4    [bisp, barr, (, ba, ), ,, dom, luiz, flávi, ca...
Name: stems, dtype: object


# Construção dos modelos de representação

## TF-IDF - fred

In [ ]:
def TF_IDF():
    tf_vect = TfidfVectorizer ( ngram_range =(1 ,2) , min_df =3 ,
    max_df =0.5 , max_features =500)
    p_tf = tf_vect . fit_transform ( docs )

## CountVectorizer -didi

In [125]:
def count_vec(train,val,test):
    c_vect = CountVectorizer(token_pattern=r"(?u)\b\w+\b|[^\w\s]", ngram_range=(1,1), strip_accents='unicode',max_features=1000)

    #juntar os tokens matizados em frases novvamente 
    text_train = train['stems'].apply(lambda tokens: ' '.join(tokens))
    text_val   = val['stems'].apply(lambda tokens: ' '.join(tokens))
    text_test = test['stems'].apply(lambda tokens: ' '.join(tokens))
    
    #print("Text train:\n", text_train.head())

    # Transformar em vetores
    X_train = c_vect.fit_transform(text_train)
    X_val   = c_vect.transform(text_val)
    X_test  = c_vect.transform(text_test)

    return X_train, X_val, X_test

In [126]:
X_train_count_vec, X_val_count_vec, X_test_count_vec = count_vec(train,val,test)

## Glove -fred

## WORD2VEC -anaana

fontes : 
-   https://sites.google.com/view/nilc-usp/resources-and-tools?authuser=0
-   https://huggingface.co/nilc-nlp/word2vec-skip-gram-300d

In [154]:
#%pip install huggingface_hub
#%pip install hf_hub_download
#%pip install safetensors
from huggingface_hub import hf_hub_download
from safetensors.numpy import load_file

def carregar_word2vec():
    path = hf_hub_download(repo_id="nilc-nlp/word2vec-skip-gram-300d",
                           filename="embeddings.safetensors")

    data = load_file(path)
    vectors = data["embeddings"]

    vocab_path = hf_hub_download(repo_id="nilc-nlp/word2vec-skip-gram-300d",
                                 filename="vocab.txt")
    with open(vocab_path) as f:
        vocab = [w.strip() for w in f]

    print(vectors.shape)

    model = KeyedVectors(vector_size=vectors.shape[1])
    model.add_vectors(vocab, vectors)
    model.fill_norms()

    return model

def vetor_frase(model, frase):
    vectors = [model[w] for w in frase if w in model]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

def word2vec(train, val, test, coluna, model):
    X_train = np.vstack([vetor_frase(model, tokens) for tokens in train[coluna]])
    X_val   = np.vstack([vetor_frase(model, tokens) for tokens in val[coluna]])
    X_test  = np.vstack([vetor_frase(model, tokens) for tokens in test[coluna]])


    return X_train, X_val, X_test

In [ ]:
model = carregar_word2vec()
X_train_word2vec, X_val_word2vec, X_test_word2vec = word2vec(train, val, test, 'stems', model)


(929606, 300)


In [ ]:
'''
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
certos = (test_df['classe'] == test_df['Previsto4']).sum()
total = len(test_df)

matriz = confusion_matrix(test_df['classe'], test_df['Previsto4'], labels=classes)

disp = ConfusionMatrixDisplay(confusion_matrix=matriz, display_labels=classes)
disp.plot()
plt.title("Matriz de confusão do teste")
plt.show()

from sklearn.metrics import classification_report

report = classification_report(test_df['classe'], test_df['Previsto4'], labels=classes, digits=4)
print(report)
'''

# Aplicar modelos de classificação

## Naive Bayes 

-   Multinomial

In [129]:
def Naive_Bayes_MN(X_train, y_train, X_val, y_val, X_test, alphas):
    best_alpha = None
    best_f1 = 0

    # Procurar o melhor alpha com o conjunto de validação
    for alpha in alphas:
        clf = MultinomialNB(alpha=alpha)
        clf.fit(X_train, y_train)
        y_val_pred = clf.predict(X_val)
        f1 = f1_score(y_val, y_val_pred, average='weighted')
        if f1 > best_f1:
            best_f1 = f1
            best_alpha = alpha
    
    print(f"Melhor alpha encontrado: {best_alpha} (F1 val = {best_f1:.4f})")

    # Treinar o modelo final 
    final_clf = MultinomialNB(alpha=best_alpha)
    final_clf.fit(X_train, y_train)

    y_test_pred = final_clf.predict(X_test)

    return y_test_pred 

In [130]:
from sklearn.metrics import f1_score, confusion_matrix, classification_report

def objective_naive_GS(trial, X_train, X_val, y_train, y_val):
    # Definir o hiperparâmetro a otimizar
    var_smoothing = trial.suggest_float("var_smoothing", 1e-12, 1e-3, log=True)
    # Criar o modelo
    model = GaussianNB(var_smoothing=var_smoothing)
    # Treinar
    model.fit(X_train, y_train)
    # Avaliar no conjunto de validação
    y_pred = model.predict(X_val)
    f1 = f1_score(y_val, y_pred, average="macro")

    return f1

def naive_bayes_gs(X_train, X_val, X_test, y_train, y_val, n_trials=30):
    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective_naive_GS(trial, X_train, X_val, y_train, y_val),
                   n_trials=n_trials, n_jobs=-1, show_progress_bar=False)
    print("Melhores hiperparâmetros encontrados:")
    print(study.best_params)
    # Treinar o melhor modelo com os melhores parâmetros
    best_model = GaussianNB(**study.best_params)
    best_model.fit(X_train, y_train)
    # Avaliar no conjunto de teste
    y_test_pred = best_model.predict(X_test)

    return y_test_pred

In [131]:
# Count Vectorizer
print ("----COUNT VECTORIZER ----")
count_vec_predicted_NB= Naive_Bayes_MN(X_train_count_vec, y_train, X_val_count_vec,y_val , X_test_count_vec, 
                                    alphas=[0.01, 0.1, 0.5, 1.0, 3.0,5.0, 6.0, 7.0]) 
lista_y = [str(c) for c in (train['classe'].unique())]
print(metrics.classification_report(
y_test, count_vec_predicted_NB, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, count_vec_predicted_NB))


#TF-IDF

#Word2vec
print ("----WORD2VEC ----")
word2vec_predicted_NB= naive_bayes_gs(X_train_word2vec, X_val_word2vec, X_test_word2vec, y_train, y_val, n_trials=30) 
lista_y = [str(c) for c in (train['classe'].unique())]
print(metrics.classification_report(
y_test, word2vec_predicted_NB, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, word2vec_predicted_NB))
#Glove


[I 2025-11-10 18:18:57,000] A new study created in memory with name: no-name-99ae1769-4fe2-45bb-b203-74b992b3ff05
[I 2025-11-10 18:18:57,137] Trial 0 finished with value: 0.4016961525191454 and parameters: {'var_smoothing': 5.082892512190099e-05}. Best is trial 0 with value: 0.4016961525191454.


----COUNT VECTORIZER ----
Melhor alpha encontrado: 0.1 (F1 val = 0.7908)
              precision    recall  f1-score   support

          -1       0.70      0.71      0.71       417
           0       0.83      0.85      0.84      1273
           1       0.26      0.20      0.22       168

    accuracy                           0.76      1858
   macro avg       0.60      0.59      0.59      1858
weighted avg       0.75      0.76      0.75      1858

[[ 296  110   11]
 [ 107 1084   82]
 [  19  116   33]]
----WORD2VEC ----


[I 2025-11-10 18:18:57,145] Trial 2 finished with value: 0.4016961525191454 and parameters: {'var_smoothing': 1.246053000786077e-09}. Best is trial 0 with value: 0.4016961525191454.
[I 2025-11-10 18:18:57,145] Trial 3 finished with value: 0.4016961525191454 and parameters: {'var_smoothing': 4.318501231483148e-05}. Best is trial 0 with value: 0.4016961525191454.
[I 2025-11-10 18:18:57,189] Trial 7 finished with value: 0.4016961525191454 and parameters: {'var_smoothing': 6.071240546005525e-10}. Best is trial 0 with value: 0.4016961525191454.
[I 2025-11-10 18:18:57,193] Trial 6 finished with value: 0.4016961525191454 and parameters: {'var_smoothing': 1.3777389060348885e-08}. Best is trial 0 with value: 0.4016961525191454.
[I 2025-11-10 18:18:57,193] Trial 1 finished with value: 0.4016961525191454 and parameters: {'var_smoothing': 2.7533810121950913e-05}. Best is trial 0 with value: 0.4016961525191454.
[I 2025-11-10 18:18:57,229] Trial 5 finished with value: 0.4016961525191454 and paramete

Melhores hiperparâmetros encontrados:
{'var_smoothing': 5.082892512190099e-05}
              precision    recall  f1-score   support

          -1       0.44      0.42      0.43       417
           0       0.84      0.32      0.47      1273
           1       0.11      0.62      0.18       168

    accuracy                           0.37      1858
   macro avg       0.46      0.46      0.36      1858
weighted avg       0.68      0.37      0.43      1858

[[175  44 198]
 [196 410 667]
 [ 30  33 105]]


## SVM

In [132]:
def objective(trial, X_train, X_val, y_train, y_val):

    #Definir hiperparâmetros a testar 
    C = trial.suggest_float('C', 0.01, 10.0, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])
    gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])
    degree = trial.suggest_int('degree', 2, 5) if kernel == 'poly' else 3

    #Criar modelo SVM 
    model = SVC(C=C, kernel=kernel, gamma=gamma, degree=degree)
    model = SVC(C=C, kernel=kernel, gamma=gamma, degree=degree)

    #treinar modelo com conjunto de treino
    model.fit(X_train, y_train)

    #prever no conjunto de validação
    y_pred = model.predict(X_val)

    #calcular f1
    f1 = f1_score(y_val, y_pred, average='macro')

    return f1

def svm(X_train, X_val, X_test, y_train, y_val, n_trials=30):
    # Chamar o Optuna para otimizar os parâmetros
    objeto_para_otimizar = optuna.create_study(direction='maximize')
    objeto_para_otimizar.optimize(lambda trial: objective(trial, X_train, X_val, y_train, y_val),
                   n_trials=n_trials, n_jobs=-1, show_progress_bar=False )

    print("Melhores hiperparâmetros encontrados:")
    print(objeto_para_otimizar.best_params)

    # Treinar o melhor modelo
    best_model = SVC(**objeto_para_otimizar.best_params)
    best_model.fit(X_train, y_train)

    #  Avaliar no conjunto de teste
    y_pred_test = best_model.predict(X_test)

    return y_pred_test

In [133]:
print ("----COUNT VECTORIZER ----")
count_vec_predicted_SVM= svm(X_train_count_vec, X_val_count_vec, X_test_count_vec, y_train, y_val, n_trials=30) 
lista_y = [str(c) for c in (train['classe'].unique())]
print(metrics.classification_report(
y_test, count_vec_predicted_SVM, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, count_vec_predicted_SVM))

print ("----WORD2VEC ----")
word2vec_predicted_SVM= svm(X_train_word2vec, X_val_word2vec, X_test_word2vec, y_train, y_val, n_trials=30) 
lista_y = [str(c) for c in (train['classe'].unique())]
print(metrics.classification_report(
y_test, word2vec_predicted_SVM, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, word2vec_predicted_SVM))

[I 2025-11-10 18:19:04,332] A new study created in memory with name: no-name-ed658ec3-9672-42c4-8427-e919848f86b1


----COUNT VECTORIZER ----


[I 2025-11-10 18:19:05,970] Trial 6 finished with value: 0.6096203833022967 and parameters: {'C': 0.28960457655521643, 'kernel': 'linear', 'gamma': 'scale'}. Best is trial 6 with value: 0.6096203833022967.
[I 2025-11-10 18:19:06,801] Trial 1 finished with value: 0.27104722792607805 and parameters: {'C': 0.3773266272321775, 'kernel': 'poly', 'gamma': 'auto', 'degree': 5}. Best is trial 6 with value: 0.6096203833022967.
[I 2025-11-10 18:19:06,981] Trial 7 finished with value: 0.6316199330354496 and parameters: {'C': 4.044673626252334, 'kernel': 'rbf', 'gamma': 'scale'}. Best is trial 7 with value: 0.6316199330354496.
[I 2025-11-10 18:19:07,234] Trial 2 finished with value: 0.5836072700685212 and parameters: {'C': 4.697652840441719, 'kernel': 'linear', 'gamma': 'auto'}. Best is trial 7 with value: 0.6316199330354496.
[I 2025-11-10 18:19:07,589] Trial 0 finished with value: 0.27104722792607805 and parameters: {'C': 1.2548726858251946, 'kernel': 'poly', 'gamma': 'auto', 'degree': 2}. Best i

Melhores hiperparâmetros encontrados:
{'C': 8.617765162510498, 'kernel': 'rbf', 'gamma': 'scale'}


[I 2025-11-10 18:19:18,598] A new study created in memory with name: no-name-41a8a2fd-e67e-419c-a23f-0ca4d791b8f8


              precision    recall  f1-score   support

          -1       0.81      0.64      0.72       417
           0       0.81      0.93      0.86      1273
           1       0.18      0.06      0.09       168

    accuracy                           0.79      1858
   macro avg       0.60      0.54      0.56      1858
weighted avg       0.75      0.79      0.76      1858

[[ 267  142    8]
 [  48 1186   39]
 [  14  144   10]]
----WORD2VEC ----


[I 2025-11-10 18:19:21,483] Trial 1 finished with value: 0.27104722792607805 and parameters: {'C': 0.9134051634912592, 'kernel': 'poly', 'gamma': 'auto', 'degree': 4}. Best is trial 1 with value: 0.27104722792607805.
[I 2025-11-10 18:19:22,367] Trial 7 finished with value: 0.27104722792607805 and parameters: {'C': 0.029499288349228252, 'kernel': 'poly', 'gamma': 'scale', 'degree': 2}. Best is trial 1 with value: 0.27104722792607805.
[I 2025-11-10 18:19:23,082] Trial 6 finished with value: 0.2886446886446887 and parameters: {'C': 0.040042962157991364, 'kernel': 'poly', 'gamma': 'scale', 'degree': 4}. Best is trial 6 with value: 0.2886446886446887.
[I 2025-11-10 18:19:23,221] Trial 5 finished with value: 0.27104722792607805 and parameters: {'C': 0.6762240290395363, 'kernel': 'rbf', 'gamma': 'auto'}. Best is trial 6 with value: 0.2886446886446887.
[I 2025-11-10 18:19:23,342] Trial 2 finished with value: 0.39928353017349344 and parameters: {'C': 0.6212884970426599, 'kernel': 'rbf', 'gamma'

Melhores hiperparâmetros encontrados:
{'C': 8.984260798339792, 'kernel': 'rbf', 'gamma': 'scale'}
              precision    recall  f1-score   support

          -1       0.59      0.44      0.51       417
           0       0.76      0.90      0.83      1273
           1       0.33      0.08      0.13       168

    accuracy                           0.72      1858
   macro avg       0.56      0.48      0.49      1858
weighted avg       0.69      0.72      0.69      1858

[[ 185  225    7]
 [ 104 1147   22]
 [  22  132   14]]


## KNN

In [156]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

def KNN(X_train, y_train, X_val, y_val, X_test):
    k_values = range(1, 200,10)  # Determinar  intervalo de valores a testar para k
    best_k = 1
    best_accuracy = 0
    
    k_list = []
    accuracy_list = [] 

    for k in k_values:

        knn = KNeighborsClassifier(n_neighbors=k)  
        knn.fit(X_train, y_train)

        y_pred_KNN = knn.predict(X_val) 
        accuracy = accuracy_score(y_val, y_pred_KNN)  

        k_list.append(k)
        accuracy_list.append(accuracy)  

        if accuracy > best_accuracy: 
            best_accuracy = accuracy
            best_k = k

    knn = KNeighborsClassifier(n_neighbors=best_k)  
    knn.fit(X_train, y_train)

    y_pred_KNN= knn.predict(X_test) 
    print(f"Best k: {best_k}")

    return y_pred_KNN


In [158]:
# Count Vectorizer
print ("----COUNT VECTORIZER ----")
count_vec_predicted_KNN= KNN(X_train_count_vec, y_train, X_val_count_vec, y_val, X_test_count_vec) 
lista_y = [str(c) for c in (train['classe'].unique())]
print(metrics.classification_report(
y_test, count_vec_predicted_KNN, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, count_vec_predicted_KNN))

#TF-IDF

#Word2vec
print ("----WORD2VEC ----")
word2vec_predicted_KNN= KNN(X_train_word2vec, y_train, X_val_word2vec, y_val, X_test_word2vec) 
lista_y = [str(c) for c in (train['classe'].unique())]
print(metrics.classification_report(
y_test, word2vec_predicted_KNN, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, word2vec_predicted_KNN))

#Glove


----COUNT VECTORIZER ----
Best k: 41
              precision    recall  f1-score   support

          -1       0.72      0.60      0.66       417
           0       0.79      0.93      0.85      1273
           1       0.00      0.00      0.00       168

    accuracy                           0.77      1858
   macro avg       0.50      0.51      0.50      1858
weighted avg       0.70      0.77      0.73      1858

[[ 250  167    0]
 [  86 1187    0]
 [  10  158    0]]
----WORD2VEC ----


c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


AttributeError: 'NoneType' object has no attribute 'split'

## Decison Tree

In [78]:
#Criar um classificador de árvore de decisão baseado em entropia
from sklearn.tree import DecisionTreeClassifier

def decision_tree(X_train, y_train, X_val, y_val, X_test):
    alphas = np.linspace(0.000, 0.01, 20)  # Testar 20 valores entre 0 e 0.01
    best_alpha = 0
    best_acc = 0

    for alpha in alphas:
        clf = DecisionTreeClassifier(criterion='entropy', ccp_alpha=alpha, random_state=42)
        clf.fit(X_train, y_train)
        y_pred_val = clf.predict(X_val)
        acc = accuracy_score(y_val, y_pred_val)

        if acc > best_acc:
            best_acc = acc
            best_alpha = alpha

    print(f"Melhor alpha: {best_alpha:.4f} ")

    # Treinar modelo final com o melhor alpha
    final_clf = DecisionTreeClassifier(criterion='entropy', ccp_alpha=best_alpha, random_state=42)
    final_clf.fit(X_train, y_train)
    y_pred_test = final_clf.predict(X_test)

    return y_pred_test

In [79]:
# Count Vectorizer
print ("----COUNT VECTORIZER ----")
count_vec_predicted_DT= decision_tree(X_train_count_vec, y_train, X_val_count_vec, y_val, X_test_count_vec) 
lista_y = [str(c) for c in (train['classe'].unique())]
print(metrics.classification_report(
y_test, count_vec_predicted_DT, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, count_vec_predicted_DT))

#TF-IDF

#Word2vec
print ("----COUNT VECTORIZER ----")
word2vec_predicted_DT= decision_tree(X_train_word2vec, y_train, X_val_word2vec, y_val, X_test_word2vec) 
lista_y = [str(c) for c in (train['classe'].unique())]
print(metrics.classification_report(
y_test, word2vec_predicted_DT, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, word2vec_predicted_DT))

#Glove


----COUNT VECTORIZER ----
Melhor alpha: 0.0005 
              precision    recall  f1-score   support

          -1       0.66      0.64      0.65       417
           0       0.81      0.82      0.81      1273
           1       0.19      0.17      0.18       168

    accuracy                           0.72      1858
   macro avg       0.55      0.54      0.55      1858
weighted avg       0.72      0.72      0.72      1858

[[ 268  132   17]
 [ 120 1047  106]
 [  19  121   28]]
----COUNT VECTORIZER ----
Melhor alpha: 0.0000 
              precision    recall  f1-score   support

          -1       0.36      0.33      0.34       417
           0       0.72      0.75      0.74      1273
           1       0.11      0.10      0.10       168

    accuracy                           0.60      1858
   macro avg       0.40      0.39      0.39      1858
weighted avg       0.58      0.60      0.59      1858

[[137 251  29]
 [213 957 103]
 [ 31 121  16]]
